In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

D:\Users\AKSHAY\Anaconda3\lib\site-packages\h5py\__init__\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7663447328205086064
]


In [2]:
def cleanString(incomingString):
    newstring = incomingString
    
#     newstring = newstring.replace("!","")
#     newstring = newstring.replace("@","")
#     newstring = newstring.replace("#","")
#     newstring = newstring.replace("$","")
#     newstring = newstring.replace("%","")
#     newstring = newstring.replace("^","")
#     newstring = newstring.replace("&","and")
#     newstring = newstring.replace("*","")
#     newstring = newstring.replace("(","")
#     newstring = newstring.replace(")","")
#     newstring = newstring.replace("+","")
#     newstring = newstring.replace("=","")
#     newstring = newstring.replace("?","")
#     newstring = newstring.replace("\'","")
#     newstring = newstring.replace("\"","")
#     newstring = newstring.replace("{","")
#     newstring = newstring.replace("}","")
#     newstring = newstring.replace("[","")
#     newstring = newstring.replace("]","")
#     newstring = newstring.replace("<","")
#     newstring = newstring.replace(">","")
#     newstring = newstring.replace("~","")
#     newstring = newstring.replace("`","")
#     newstring = newstring.replace(":","")
#     newstring = newstring.replace(";","")
#     newstring = newstring.replace("|","")
#     newstring = newstring.replace("\\","")
#     newstring = newstring.replace("/","")
#     newstring = newstring.replace(",","")
#     newstring = newstring.replace(".","")
    return newstring

In [3]:
import itertools
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

import keras
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras import utils

config = tf.ConfigProto( device_count = {'GPU': 3000 , 'CPU': 2} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

Using TensorFlow backend.


In [4]:
print("Tensorflow version", tf.__version__)

Tensorflow version 1.7.0


#### loading the data

In [10]:
data = pd.read_csv('E:\\ML\\Unsupervised\\datapolarity.csv',encoding = 'ISO-8859-1')

In [11]:
from sklearn.utils import shuffle
data = shuffle(data)

In [12]:
data.columns

Index(['Actual Title', 'Cleaned Title', 'Compound', 'Negative', 'Neutral',
       'Positive', 'Date'],
      dtype='object')

#### Split the data into train and test

In [13]:
train_size = int(len(data)*.80)
print('Train size:', train_size)

Train size: 622511


In [34]:
train_sentence = data['Actual Title'][:train_size]
train_sentence = [x for x in train_sentence]
train_posts = data['Cleaned Title'][:train_size]
train_posts = [cleanString(x) for x in train_posts] 
train_tags = data['Compound'][:train_size]  
train_tags = [round(x,1)  for x in train_tags]
train_time = data['Date'][:train_size]  
train_time = [x for x in train_time]

train_sentence1 = []
train_tags1 = []
train_posts1 = []
train_time1=[]

for i in range(len(train_tags)):
    if train_tags[i] < -0.3:
        train_sentence1.append(train_sentence[i])
        train_tags1.append(-1)
        train_posts1.append(str(train_posts[i]))
        train_time1.append(train_time[i])
    elif train_tags[i] >0.3:
        train_sentence1.append(train_sentence[i])
        train_tags1.append(1)
        train_posts1.append(str(train_posts[i]))
        train_time1.append(train_time[i])

test_sentence = data['Actual Title'][train_size:]
test_sentence = [x for x in test_sentence]
test_posts = data['Cleaned Title'][train_size:]
test_posts = [cleanString(x) for x in test_posts]
test_tags = data['Compound'][train_size:]
test_tags = [round(x,1)  for x in test_tags] 
test_time = data['Date'][train_size:]
test_time = [x for x in test_time]
test_sentence1 = []
test_tags1 = []
test_posts1 = []
test_time1 = []
for i in range(len(test_tags)):
    if test_tags[i] < -0.3:
        test_sentence1.append(test_sentence[i])
        test_tags1.append(-1)
        test_posts1.append(str(test_posts[i]))
        test_time1.append(test_time[i])
    elif test_tags[i] >0.3:
        test_sentence1.append(test_sentence[i])
        test_tags1.append(1)
        test_posts1.append(str(test_posts[i]))
        test_time1.append(test_time[i])
        
for x in test_tags1[:10]:
    print (x )
    
print (len(test_tags1),len(train_tags1),sep="\n")

1
1
-1
1
-1
1
1
1
-1
1
40981
163154


In [35]:
for i in range(10):
    print(test_tags1[i])

1
1
-1
1
-1
1
1
1
-1
1


### tokenize the data

In [36]:
from keras.preprocessing import text

In [37]:
max_words = 3000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

In [38]:
tokenize.fit_on_texts(train_posts1)
x_train = tokenize.texts_to_matrix(train_posts1)
x_test = tokenize.texts_to_matrix(test_posts1)

In [39]:
len(x_test[40])

3000

### Use sklearn utility to convert label strings to numbered index


In [40]:
encoder = LabelEncoder()
encoder.fit(train_tags1)

LabelEncoder()

In [41]:
y_train = encoder.transform(train_tags1)
y_test = encoder.transform(test_tags1)

In [42]:
y_train

array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [43]:
y_test

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [44]:
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [45]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (163154, 3000)
x_test shape: (40981, 3000)
y_train shape: (163154, 2)
y_test shape: (40981, 2)


In [46]:
batch_size = 32

In [47]:
### Build the model
model = Sequential()
model.add(Dense(512,input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])


In [48]:
history = model.fit(x_train,y_train,
                   batch_size=batch_size,
                   epochs=100,
                   verbose=1,
                   validation_split=0.2)

Train on 130523 samples, validate on 32631 samples
Epoch 1/100
130523/130523 [==============================] - 162s 1ms/step - loss: 0.0534 - acc: 0.9789 - val_loss: 0.0221 - val_acc: 0.9915
Epoch 2/100
130523/130523 [==============================] - 156s 1ms/step - loss: 0.0187 - acc: 0.9932 - val_loss: 0.0201 - val_acc: 0.9923
Epoch 3/100
130523/130523 [==============================] - 158s 1ms/step - loss: 0.0157 - acc: 0.9943 - val_loss: 0.0202 - val_acc: 0.9924
Epoch 4/100
130523/130523 [==============================] - 155s 1ms/step - loss: 0.0133 - acc: 0.9952 - val_loss: 0.0202 - val_acc: 0.9933
Epoch 5/100
130523/130523 [==============================] - 157s 1ms/step - loss: 0.0122 - acc: 0.9957 - val_loss: 0.0219 - val_acc: 0.9926
Epoch 6/100
130523/130523 [==============================] - 156s 1ms/step - loss: 0.0110 - acc: 0.9960 - val_loss: 0.0212 - val_acc: 0.9931
Epoch 7/100
130523/130523 [==============================] - 159s 1ms/step - loss: 0.0104 - acc: 0.9963

130523/130523 [==============================] - 150s 1ms/step - loss: 0.0023 - acc: 0.9995 - val_loss: 0.0457 - val_acc: 0.9932
Epoch 59/100
130523/130523 [==============================] - 152s 1ms/step - loss: 0.0023 - acc: 0.9995 - val_loss: 0.0458 - val_acc: 0.9936
Epoch 60/100
130523/130523 [==============================] - 156s 1ms/step - loss: 0.0023 - acc: 0.9996 - val_loss: 0.0500 - val_acc: 0.9928
Epoch 61/100
130523/130523 [==============================] - 150s 1ms/step - loss: 0.0025 - acc: 0.9995 - val_loss: 0.0473 - val_acc: 0.9936
Epoch 62/100
130523/130523 [==============================] - 149s 1ms/step - loss: 0.0027 - acc: 0.9995 - val_loss: 0.0472 - val_acc: 0.9935
Epoch 63/100
130523/130523 [==============================] - 149s 1ms/step - loss: 0.0021 - acc: 0.9996 - val_loss: 0.0476 - val_acc: 0.9932
Epoch 64/100
130523/130523 [==============================] - 149s 1ms/step - loss: 0.0023 - acc: 0.9996 - val_loss: 0.0480 - val_acc: 0.9933
Epoch 65/100
130523

In [49]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

40981/40981 [==============================] - 6s 148us/step
Test score: 0.05468302434844786
Test accuracy: 0.9935335887362436


In [54]:
text_labels = encoder.classes_ 

for i in range(1000):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print('Actual Title: '+test_sentence1[i])
    print('Cleaned Title: '+test_posts1[i])
    print('Time: ' + str(test_time1[i]))
    print('Actual label:' + str(test_tags1[i]))
    print("Predicted label: " + str(predicted_label) + "\n")

Actual Title: I have had more failure than success: Shikhar Dhawan
Cleaned Title: failur success shikhar dhawan
Time: 01-01-1970 05:30
Actual label:1
Predicted label: 1

Actual Title: Employees to pay tax on gifts worth more than â¹50,000
Cleaned Title: employe pay tax gift worth
Time: 05-02-2017 21:36
Actual label:1
Predicted label: 1

Actual Title: No threat of systemic risk from housing sector: RBI Report
Cleaned Title: threat system risk hous sector rbi report
Time: 28-06-2016 22:16
Actual label:-1
Predicted label: -1

Actual Title: AIADMK crisis LIVE: 19 MLAs loyal to Dinakaran withdraw support to CM Palaniswami
Cleaned Title: aiadmk crisi live mlas loyal dinakaran withdraw support cm palaniswami
Time: nan
Actual label:1
Predicted label: 1

Actual Title: Crude oil futures climb following attack on Libya's main oil terminals
Cleaned Title: crude oil futur climb follow attack libya main oil termin
Time: 26-12-2014 11:51
Actual label:-1
Predicted label: -1

Actual Title: Abbott like

Actual Title: BOJ shocks markets with more easing as inflation slows
Cleaned Title: boj shock market eas inflat slow
Time: 31-10-2014 12:24
Actual label:-1
Predicted label: -1

Actual Title: ['Business remains dull as Muzaffarnagar limps back to normalcy']
Cleaned Title: busi remain dull muzaffarnagar limp back normalci
Time: 12-09-2013 18:47
Actual label:-1
Predicted label: -1

Actual Title: ['Lok Sabha passes anti-rape Bill, majority of MPs absent during vote']
Cleaned Title: lok sabha pass anti rape bill major mps absent vote
Time: 19-03-2013 21:03
Actual label:-1
Predicted label: -1

Actual Title: Sensex ends below 21000 on Ukraine woes; Sun, BHEL slip 3%
Cleaned Title: sensex end ukrain woe sun bhel slip
Time: 03-03-2014 22:09
Actual label:-1
Predicted label: -1

Actual Title: Can Fin Homes surges on strong Q3 results
Cleaned Title: fin home surg strong q result
Time: 22-01-2016 14:50
Actual label:1
Predicted label: 1

Actual Title: ['Sell Wockhardt with a stop loss of Rs 1,070: A

Actual Title: ['Recipe: This will be the best lamb lasagne to ever adorn your dining table']
Cleaned Title: recip best lamb lasagn ever adorn dine tabl
Time: 17-05-2017 18:14
Actual label:1
Predicted label: 1

Actual Title: ['Mu Sigma attracts a clutch of foreign investors, gets valued at $1 billion, Dhiraj Rajaram is now king of data']
Cleaned Title: mu sigma attract clutch foreign investor get valu billion dhiraj rajaram king data
Time: 20-02-2013 10:31
Actual label:1
Predicted label: 1

Actual Title: Homebuyers of Jaypee, Amrapali projects join hands, want state-owned HUDCO, NBCC to rescue them
Cleaned Title: homebuy jaype amrapali project join hand want state own hudco nbcc rescu
Time: 17-08-2017 16:58
Actual label:1
Predicted label: 1

Actual Title: ["Arvind Kejriwal's power promises tripped? NTPC rejects Delhi govt plea not to stop supply to BSES Yamuna"]
Cleaned Title: arvind kejriw power promis trip ntpc reject delhi govt plea stop suppli bses yamuna
Time: 31-01-2014 16:05
Actu

Actual Title: Nifty above 8600, Sensex gains 250 pts; Reliance leads
Cleaned Title: nifti sensex gain pts relianc lead
Time: 22-07-2015 13:07
Actual label:1
Predicted label: 1

Actual Title: Himalayan natural mineral water founder Dadi Balsara dies at 81
Cleaned Title: himalayan natur miner water founder dadi balsara die
Time: 20-06-2014 05:55
Actual label:-1
Predicted label: -1

Actual Title: Murray wins Davis Cup for GB
Cleaned Title: murray win davi cup gb
Time: 01-01-1970 05:30
Actual label:1
Predicted label: 1

Actual Title: ['Coal Ministry denies giving any paper to scam accused']
Cleaned Title: coal ministri deni give paper scam accus
Time: 12-07-2013 07:26
Actual label:-1
Predicted label: -1

Actual Title: Sadbhav Infrastructure surges 17% after block deal
Cleaned Title: sadbhav infrastructur surg block deal
Time: 05-05-2017 11:25
Actual label:-1
Predicted label: -1

Actual Title: Wall Street set to open slightly higher on strong ADP data
Cleaned Title: wall street set open sli

Actual Title: Reliance posts impressive Q4 profit of Rs 6,381 crore on higher refining margins
Cleaned Title: relianc post impress q profit rs crore higher refin margin
Time: 17-04-2015 16:48
Actual label:1
Predicted label: 1

Actual Title: BlackRock says investors will benefit from MSCI nod on China stocks
Cleaned Title: blackrock say investor benefit msci nod china stock
Time: 21-06-2017 04:40
Actual label:1
Predicted label: 1

Actual Title: Gold firms above 3-1/2 month low after sluggish US jobs data
Cleaned Title: gold firm month low sluggish us job data
Time: 03-07-2015 10:24
Actual label:-1
Predicted label: -1

Actual Title: Better monsoon to reduce price pressure in pulses: Study
Cleaned Title: better monsoon reduc price pressur puls studi
Time: 28-06-2016 19:34
Actual label:1
Predicted label: 1

Actual Title: Dynamatic Technologies surges as company swings to profits in Q4
Cleaned Title: dynamat technolog surg compani swing profit q
Time: 30-05-2016 10:45
Actual label:1
Predict

Actual Title: US drone strike kills at least five militants in Pakistan
Cleaned Title: us drone strike kill least five milit pakistan
Time: 01-01-1970 05:30
Actual label:-1
Predicted label: -1

Actual Title: ['DLF advances 2% on reporting profit in Q1']
Cleaned Title: dlf advanc report profit q
Time: 30-08-2016 11:09
Actual label:1
Predicted label: 1

Actual Title: Wish you were listed, Patanjali Ayurved: CLSA
Cleaned Title: wish list patanjali ayurv clsa
Time: 28-08-2015 11:54
Actual label:1
Predicted label: 1

Actual Title: Election Results 2017 LIVE: Modi tsunami destroys Rahul-Akhilesh alliance in UP, Cong gets Punjab
Cleaned Title: elect result live modi tsunami destroy rahul akhilesh allianc cong get punjab
Time: 11-03-2017 09:53
Actual label:-1
Predicted label: -1

Actual Title: Yes Bank receives ratings from CARE Ratings, India Ratings for Basel III ATl Bonds; stocks marginally up
Cleaned Title: yes bank receiv rate care rate india rate basel iii atl bond stock margin
Time: 23-

Actual Title: As you get closer to your goals, move from equity to debt
Cleaned Title: get closer goal move equiti debt
Time: 26-05-2014 20:11
Actual label:-1
Predicted label: -1

Actual Title: ["Azam Khan attacks UP Governor Ram Naik for 'stalling' controversial bills"]
Cleaned Title: azam khan attack governor ram naik stall controversi bill
Time: 08-03-2016 16:55
Actual label:-1
Predicted label: -1

Actual Title: Equities erase all of 2016 gains 
Cleaned Title: equiti eras gain
Time: 21-11-2016 23:40
Actual label:1
Predicted label: 1

Actual Title: ['Is Murthy really wrong in his tirade against Infosys board?']
Cleaned Title: murthi realli wrong tirad infosi board
Time: 22-08-2017 11:30
Actual label:-1
Predicted label: -1

Actual Title: DBS optimistic about plans to expand in India
Cleaned Title: dbs optimist plan expand india
Time: 10-07-2015 19:30
Actual label:1
Predicted label: 1

Actual Title: Dish TV looks attractive from three years' perspective
Cleaned Title: dish tv look attr

Actual Title: ['Kotak Bank bullish on corporate loans, eyes up to 20% growth']
Cleaned Title: kotak bank bullish corpor loan eye growth
Time: 05-07-2015 14:19
Actual label:1
Predicted label: 1

Actual Title: Dow Breaks 17,000 Following Strong US Job Gains
Cleaned Title: dow break follow strong us job gain
Time: 03-07-2014 19:31
Actual label:1
Predicted label: 1

Actual Title: ['Data growth may offset revenue decline for Airtel, Idea and Reliance Communications in Q2']
Cleaned Title: data growth may offset revenu declin airtel idea relianc communic q
Time: 19-10-2015 08:29
Actual label:1
Predicted label: 1

Actual Title: Happy New Year sees the best opening ever
Cleaned Title: happi new year see best open ever
Time: 25-10-2014 21:45
Actual label:1
Predicted label: 1

Actual Title: Vikram Bhat: the metalhead who managed one of Bengaluru's best known vegetarian restaurants
Cleaned Title: vikram bhat metalhead manag one bengaluru best known vegetarian restaur
Time: 16-05-2015 00:29
Actual 

Actual Title: Reliance Infra gains 3%, defence arm signs MoU with UAE co
Cleaned Title: relianc infra gain defenc arm sign mou uae co
Time: 28-09-2015 15:51
Actual label:1
Predicted label: 1

Actual Title: Indian IT companies among world's worst paymasters
Cleaned Title: indian compani among world worst paymast
Time: 01-01-1970 05:30
Actual label:-1
Predicted label: -1

Actual Title: Ola offers Rs 1 lakh benefits to drivers who opt for CNG
Cleaned Title: ola offer rs lakh benefit driver opt cng
Time: 01-01-1970 05:30
Actual label:1
Predicted label: 1

Actual Title: Economy on strong wicket, may grow at 7.5%: Aditya Puri
Cleaned Title: economi strong wicket may grow aditya puri
Time: 22-09-2016 08:30
Actual label:1
Predicted label: 1

Actual Title: ['Cox & Kings Q3 profit jumps 10 fold to Rs 30.91 crore']
Cleaned Title: cox king q profit jump fold rs crore
Time: 14-02-2014 17:08
Actual label:1
Predicted label: 1

Actual Title: 8K Miles Software Q2 profit rises 31.4% to Rs Rs 4.14 cr
Cle

Actual Title: Volatile crude to put pressure on RIL's gross refining margins
Cleaned Title: volatil crude put pressur ril gross refin margin
Time: 23-04-2016 00:38
Actual label:-1
Predicted label: -1

Actual Title: Millennial Indians Travel Abroad More, Favour Singapore, US: Survey
Cleaned Title: millenni indian travel abroad favour singapor us survey
Time: 02-12-2015 21:08
Actual label:1
Predicted label: 1

Actual Title: Royal Enfield Sales Help Eicher Motors Post 40% Jump in Q2 Net
Cleaned Title: royal enfield sale help eicher motor post jump q net
Time: 22-07-2015 11:24
Actual label:1
Predicted label: 1

Actual Title: ['SAIL: Project delays, wage bill could hurt co']
Cleaned Title: sail project delay wage bill could hurt co
Time: 21-02-2013 04:40
Actual label:-1
Predicted label: -1

Actual Title: Crude oil futures climb Rs 74 a barrel fuelled by global cues
Cleaned Title: crude oil futur climb rs barrel fuell global cue
Time: 16-02-2016 11:48
Actual label:-1
Predicted label: -1

Act

In [ ]:
from keras.models import load_model
model.save('model_file1.h5')
my_model = load_model('model_file1.h5')

In [ ]:
my_model

In [ ]:
test_time[1]

In [ ]:
Actual Title: Cyient Q1 profit falls 20% to Rs 75 cr, $ revenue down 2.6%
Cleaned Title: cyient q profit fall rs cr revenu
Time: 16-07-2015 17:19
Actual label:1
Predicted label: 1
    
    
